In [1]:
import json
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
from tqdm import tqdm

openai.__version__

'0.28.0'

In [2]:
openai.api_key = ''

In [3]:
openai.api_type = 'azure'
openai.api_base = 'https://husein-ai2-aiservices.openai.azure.com/'
openai.api_version = '2023-03-15-preview'

AZURE_EMBEDDING_ENGINE = 'text-embedding-ada-002'

In [4]:
contexts, questions = [], []

with open('ultrachat-lom-agc.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(8034, 8034)

In [5]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    while True:
        try:
            r = openai.Embedding.create(input=contexts[i], engine=AZURE_EMBEDDING_ENGINE)
            contexts_v.append(np.array(r.data[0]['embedding']))
            break
        except:
            time.sleep(1.0)

100%|██████████| 8034/8034 [16:35<00:00,  8.07it/s]  


In [7]:
questions_v = []
for i in tqdm(range(len(questions))):
    while True:
        try:
            r = openai.Embedding.create(input=questions[i], engine=AZURE_EMBEDDING_ENGINE)
            questions_v.append(np.array(r.data[0]['embedding']))
            break
        except:
            time.sleep(1.0)

100%|██████████| 8034/8034 [04:49<00:00, 27.79it/s]


In [8]:
contexts_v_np = np.array(contexts_v)
questions_v_np = np.array(questions_v)
contexts_v_np.shape, questions_v_np.shape

((8034, 1536), (8034, 1536))

In [9]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 8034/8034 [05:24<00:00, 24.77it/s]


In [10]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.19168533731640527
3 0.2827981080408265
5 0.322504356484939
10 0.36855862584017923
